In [ ]:
from zipfile import ZipFile
file_name = "ThreeDateFruits.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

In [ ]:
# (1) Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="whitegrid")

import os               #create folders, open ...
import glob as gb       #create folders, open ...
import cv2              #opencv (Dealing with images)
import keras 

In [ ]:
#Define the path
trainpath = '/content/'
testpath = '/content/'
predpath = '/content/'

#! cd fourDateFruits/
#! ls

In [ ]:
# (2) Open folders 
#First check The train folder to have a look to its content
for folder in  os.listdir(trainpath + 'Training_dataset') :
    files = gb.glob(pathname= str( trainpath +'Training_dataset//' + folder + '/*.jpg'))
    print(f'For training data , found {len(files)} items in folder {folder}')

In [ ]:
#The test folder
for folder in  os.listdir(testpath +'Testing_dataset') : 
    files = gb.glob(pathname= str( testpath +'Testing_dataset//' + folder + '/*.jpg'))
    print(f'For testing data , found {len(files)} items in folder {folder}')

In [ ]:
#The prediction folder
files = gb.glob(pathname= str(predpath +'Prediction_dataset/*.jpg'))
print(f'For Prediction data , found {len(files)} items')

In [ ]:
# (3) Checking Images 
#Checking images sizes
#since we have 3 categories , we first need to create a dictionary with their names & indices, 
#also create a function to get the code back {key:value} {x:y}

code = {'sukkri':0 ,'ajwa':1, 'sagei':2}
def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x

In [ ]:
#The images sizes in train folder
size = []
for folder in  os.listdir(trainpath +'Training_dataset') : 
    files = gb.glob(pathname= str( trainpath +'Training_dataset//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        size.append(image.shape)
pd.Series(size).value_counts() 

In [ ]:
#The prediction images
size = []
files = gb.glob(pathname= str(predpath +'Prediction_dataset/*.jpg'))
for file in files: 
    image = plt.imread(file)
    size.append(image.shape)
pd.Series(size).value_counts()

In [ ]:
# (4) Reading Images
#now it's time to read all images & convert it into arrays

s = 100 

X_train = []
y_train = [] 
for folder in  os.listdir(trainpath +'Training_dataset') : 
    files = gb.glob(pathname= str( trainpath +'Training_dataset//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file) #cv2 or plt #read image
        image_array = cv2.resize(image , (s,s))
        X_train.append(list(image_array))
        y_train.append(code[folder])

In [ ]:
print(f'we have {len(X_train)} items in X_train')

In [ ]:
#we have have a look to random pictures in X_train , and to adjust their title using the y value
  
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_train),36))) : 
    plt.subplot(6,6,n+1) 
    plt.imshow(X_train[i])   
    plt.axis('off')
    plt.title(getcode(y_train[i]))

In [ ]:
X_test = []
y_test = []
for folder in  os.listdir(testpath +'Testing_dataset') : 
    files = gb.glob(pathname= str(testpath + 'Testing_dataset//' + folder + '/*.jpg'))
    for file in files: 
        image = plt.imread(file)
        image_array = cv2.resize(image , (s,s)) 
        X_test.append(list(image_array))
        y_test.append(code[folder])

In [ ]:
print(f'we have {len(X_test)} items in X_test')

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_test),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_test[i])    
    plt.axis('off')
    plt.title(getcode(y_test[i]))

In [ ]:
#Also with Prediction data , without having title ofcourse
X_pred = [] 
files = gb.glob(pathname= str(predpath + 'Prediction_dataset/*.jpg'))
for file in files: 
    image = plt.imread(file)
    image_array = cv2.resize(image , (s,s))
    X_pred.append(list(image_array))

In [ ]:
print(f'we have {len(X_pred)} items in X_pred')

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')

In [ ]:
# (5) Building the Model
#First to convert the data into arrays using numpy
X_train = np.array(X_train)
X_test = np.array(X_test)
X_pred_array = np.array(X_pred)
y_train = np.array(y_train)
y_test = np.array(y_test)

#shape 

print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

In [ ]:
#Build the CNN model by Keras , using Conv2D layers , MaxPooling & Denses
# Tried relu and softmax as activation functions for Denses, but the best result came with Sigmod
# Tried sigmod for Conv2D - but best results came with relu
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(s,s,3)), #kernel_size= filter size
        keras.layers.Conv2D(150,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(120,activation='sigmod') ,    
        keras.layers.Dense(100,activation='sigmod') ,    
        keras.layers.Dense(50,activation='sigmod') , 
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(3,activation='softmax') ,   #softmax used with classification  # 3 عدد الفولدرز
        ])

In [ ]:
#now to compile the model , using adam optimizer , & sparse categorical crossentropy loss
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print('Model Details are : ')
print(KerasModel.summary())

In [ ]:
#now to train the model 
epochs = 20 
ThisModel = KerasModel.fit(X_train, y_train, epochs=epochs,batch_size=64,verbose=1)

In [ ]:
#The final loss & accuracy
#Evaluate the accuracy
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

In [ ]:
y_result = KerasModel.predict(X_pred_array)

print('Prediction Shape is {}'.format(y_result.shape))

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred[i])    
    plt.axis('off')
    plt.title(getcode(np.argmax(y_result[i])))